In [1]:
import os
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import lightning.pytorch as pl
from pytorch_lightning.loggers import WandbLogger
import segmentation_models_pytorch as smp
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.optim import AdamW
import torch.nn as nn
from lightning.pytorch.callbacks import ProgressBar
from torchmetrics.functional import dice, f1_score, jaccard_index
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
import torchvision.models as models
from sklearn.metrics import accuracy_score, roc_auc_score
from torchvision.models import resnet50, ResNet50_Weights

# config
config = {
    "wandb": True,
    'competition'   : 'rsna-atd' ,
    '_wandb_kernel' : 'johnnyhyl',
    "data_path": "../",
    "model": {
        "encoder_name": models.efficientnet_b1(weights='DEFAULT'),
        "loss_smooth": 1.0,
        "loss": nn.BCEWithLogitsLoss(),
        "optimizer_params": {"lr": 0.01, "weight_decay": 0.0},
        "scheduler": {
            "name": "CosineAnnealingLR",
            "params": {
                "CosineAnnealingLR": {"T_max": 20, "eta_min": 1e-06, "last_epoch": -1},
                "ReduceLROnPlateau": {
                    "factor": 0.316,
                    "mode": "min",
                    "patience": 3,
                    "verbose": True,
                },
            },
        },
        "seg_model": "Unet",
    },
    "output_dir": "models",
    "progress_bar_refresh_rate": 10,
    "seed": 42,
    "train_bs": 16,
    "use_aug": True,
    "trainer": {
        "enable_progress_bar": True,
        "max_epochs": 50,
        "min_epochs": 20,
        "accelerator": "mps",
        "devices": 1,
    },
    "valid_bs": 16,
    "workers": 0,
    "device": "mps",
    "folds": {
        "n_splits": 2,
        "random_state": 42,
        "train_folds": [0, 1, 2, 3]
    },
    'target_col': [ "bowel_injury", "extravasation_injury", "kidney_healthy", "kidney_low",
                   "kidney_high", "liver_healthy", "liver_low", "liver_high",
                   "spleen_healthy", "spleen_low", "spleen_high"],
}


In [2]:
BASE_PATH = '.'

# train
df = pd.read_csv(f"{BASE_PATH}/train.csv")
df["image_path"] = f"{BASE_PATH}/train_images"\
                    + "/" + df.patient_id.astype(str)\
                    + "/" + df.series_id.astype(str)\
                    + "/" + df.instance_number.astype(str) +".png"
df = df.drop_duplicates()

df.head(2)

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,...,spleen_healthy,spleen_low,spleen_high,any_injury,series_id,instance_number,injury_name,image_path,width,height
0,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,362,Active_Extravasation,./train_images/10004/21057/362.png,512,512
1,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,363,Active_Extravasation,./train_images/10004/21057/363.png,512,512


In [4]:
# count of each patient ID in df
df.patient_id.value_counts().max()

410

In [5]:
class CTDataSet(Dataset):
    def __init__(self, df, labels, transform=None):
        self.image_paths = df.image_path
        self.labels = labels
        self.transform = transform
        self.df = df

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')  # Assuming images are in RGB format
        label = torch.tensor(self.df.loc[idx, self.labels].values.astype(float), dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        # Splitting the label tensor based on your provided indices
        label_splits = (label[0:1], label[1:2], label[2:5], label[5:8], label[8:11])
        return image, label_splits

# Define transformations and augmentations
transform = transforms.Compose([
    # transforms.Resize((128, 128)),
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomVerticalFlip(),
    transforms.RandomAutocontrast(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Pad(padding = 20),
    transforms.ColorJitter(),
    transforms.RandomPerspective(),
    transforms.RandomAffine(degrees=(5, 20), translate=(0.1, 0.3), scale=(0.75, 0.9)),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# paths  = train_data.image_path.tolist()
# labels = train_data[config.TARGET_COLS].values

* All the images **should** be the same orientation.
* Applying flips might weaken the model as it can reasonably expect a certain orientation
* some people have situs inversus (rare)
* if the trauma is bad the person might not by lying straight, so there may be a clue in their having trauma from the angle of the image
* applying radom rotations might destroy this information

In [5]:
class MetricsCalculator:
    
    def __init__(self, mode = 'binary'):
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
        
        self.mode = mode
    
    def update(self, logits, target):
        """
        Update the metrics calculator with predicted values and corresponding targets.
        
        Args:
            predicted (torch.Tensor): Predicted values.
            target (torch.Tensor): Ground truth targets.
        """
        if self.mode == 'binary':
            probabilities = torch.sigmoid(logits)
            predicted = (probabilities > 0.5)
        else:
            probabilities = F.softmax(logits, dim = 1)
            predicted = torch.argmax(probabilities, dim=1)
            
        self.probabilities.extend(probabilities.detach().cpu().numpy())
        self.predictions.extend(predicted.detach().cpu().numpy())
        self.targets.extend(target.detach().cpu().numpy())
    
    def reset(self):
        """Reset the stored predictions and targets."""
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
    
    def compute_accuracy(self):
        """
        Compute the accuracy metric.
        
        Returns:
            float: Accuracy.
        """
        return accuracy_score(self.targets, self.predictions)
    
    def compute_auc(self):
        """
        Compute the AUC (Area Under the Curve) metric.
        
        Returns:
            float: AUC.
        """
        if self.mode == 'multi':
            return roc_auc_score(self.targets, self.probabilities, multi_class = 'ovo', labels=[0, 1, 2])
    
        else:
            return roc_auc_score(self.targets, self.probabilities)

In [6]:
class LightningModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config

        # self.input = nn.Conv2d(3, 3, kernel_size = 3)
        model = self.config['encoder_name']
        
        self.features = model.features
        self.avgpool = model.avgpool
        
        self.bowel = nn.Linear(1280, 1)
        self.extravasation = nn.Linear(1280, 1)
        self.kidney = nn.Linear(1280, 3)
        self.liver = nn.Linear(1280,3) 
        self.spleen = nn.Linear(1280, 3)

        self.bowel_metrics = MetricsCalculator(mode='binary')
        self.extravasation_metrics = MetricsCalculator(mode='binary')
        self.kidney_metrics = MetricsCalculator(mode='multi')
        self.liver_metrics = MetricsCalculator(mode='multi')
        self.spleen_metrics = MetricsCalculator(mode='multi')
        
        self.loss_module = nn.BCEWithLogitsLoss()
        self.val_step_outputs = []
        self.val_step_labels = []
        self.save_hyperparameters()

    def forward(self, batch):
        # extract features
        # x = self.input(batch)
        x = self.features(batch)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        
        # output logits
        bowel = self.bowel(x)
        extravasation = self.extravasation(x)
        kidney = self.kidney(x)
        liver = self.liver(x)
        spleen = self.spleen(x)
        
        return bowel, extravasation, kidney, liver, spleen

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), **self.config["optimizer_params"])

        if self.config["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config["scheduler"]["params"][self.config["scheduler"]["name"]],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config["scheduler"]["params"][self.config["scheduler"]["name"]],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def training_step(self, batch, batch_idx):
        imgs, (bowel_labels, extravasation_labels, kidney_labels, liver_labels, spleen_labels) = batch
        bowel, extravasation, kidney, liver, spleen = self(imgs)
        
        # Combine all the labels and predictions into single tensors
        all_labels = torch.cat((bowel_labels, extravasation_labels, kidney_labels, liver_labels, spleen_labels), dim=1)
        all_preds = torch.cat((bowel, extravasation, kidney, liver, spleen), dim=1)
        
        # Compute loss
        loss = self.loss_module(all_preds, all_labels.float())
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=16)
        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        imgs, (bowel_labels, extravasation_labels, kidney_labels, liver_labels, spleen_labels) = batch
        bowel, extravasation, kidney, liver, spleen = self(imgs)
        
        # Combine all the labels and predictions into single tensors
        all_labels = torch.cat((bowel_labels, extravasation_labels, kidney_labels, liver_labels, spleen_labels), dim=1)
        all_preds = torch.cat((bowel, extravasation, kidney, liver, spleen), dim=1)
        
        # Compute loss
        loss = self.loss_module(all_preds, all_labels.float())
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)

    # def on_validation_epoch_end(self):
    #     # all_preds = torch.cat(self.val_step_outputs)
    #     # all_labels = torch.cat(self.val_step_labels)
    #     # self.val_step_outputs.clear()
    #     # self.val_step_labels.clear()
    #     # val_dice = dice(all_preds, all_labels.long())
    #     # val_f1 = f1_score(all_preds.sigmoid(), all_labels.long(), task = "binary")
    #     val_iou = jaccard_index(num_classes=2, task='binary', preds=all_preds.sigmoid(), target=all_labels.long())
    #     self.log("val_iou", val_iou, on_step=False, on_epoch=True, prog_bar=True)

    def on_epoch_end(self, outputs):
        # Compute and log metrics
        self.log('train_bowel_accuracy', self.bowel_metrics.compute_accuracy())
        self.log('train_bowel_auc', self.bowel_metrics.compute_auc())
        self.log('train_extravasation_accuracy', self.extravasation_metrics.compute_accuracy())
        self.log('train_extravasation_auc', self.extravasation_metrics.compute_auc())
        # ... and similarly for the other tasks ...
        # Reset metrics
        self.bowel_metrics.reset()
        self.extravasation_metrics.reset()
        self.kidney_metrics.reset()
        self.liver_metrics.reset()
        self.spleen_metrics.reset()

In [7]:
import warnings
import gc

warnings.filterwarnings("ignore")

import os
import torch
import pandas as pd
import lightning.pytorch as pl
from pprint import pprint
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from torch import mps
from sklearn.model_selection import StratifiedGroupKFold

df['stratify'] = ''
for col in config['target_col']:
    df['stratify'] += df[col].astype(str)

df = df.reset_index(drop=True)
skf = StratifiedGroupKFold(n_splits=config['folds']['n_splits'], shuffle=True, random_state=config['seed'])
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['stratify'], df["patient_id"])):
    df.loc[val_idx, 'fold'] = fold

    train_df = df.iloc[train_idx]
    valid_df = df.iloc[val_idx]

    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)

    dataset_train = CTDataSet(train_df, labels=config['target_col'], transform=transform)
    dataset_validation = CTDataSet(valid_df, labels=config['target_col'], transform=transform)

    data_loader_train = DataLoader(
        dataset_train, batch_size=config["train_bs"], shuffle=True, num_workers=config["workers"]
    )
    data_loader_validation = DataLoader(
        dataset_validation, batch_size=config["valid_bs"], shuffle=True, num_workers=config["workers"]
    )

    pl.seed_everything(config["seed"])

    filename = f"model_fold_{fold}"

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=config["output_dir"],
        mode="min",
        filename=filename,
        save_top_k=1,
        verbose=1,
    )

    progress_bar_callback = TQDMProgressBar(refresh_rate=config["progress_bar_refresh_rate"])

    early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1)

    wandb_logger = WandbLogger(log_model="all")

    trainer = pl.Trainer(
        callbacks=[checkpoint_callback, early_stop_callback, progress_bar_callback], logger=wandb_logger, **config["trainer"]
    )

    model = LightningModule(config['model'])

    trainer.fit(model, data_loader_train, data_loader_validation)
    torch.save(model.state_dict(), os.path.join(config["output_dir"], f"{filename}.pth"))
    model = model.to('cpu')
    mps.empty_cache()
    del (
        dataset_train,
        dataset_validation,
        train_df,
        valid_df,
        data_loader_train,
        data_loader_validation,
        model,
        trainer,
        checkpoint_callback,
        progress_bar_callback,
        early_stop_callback,
    )
    mps.empty_cache()
    gc.collect()

Global seed set to 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnny-hyland. Use `wandb login --relogin` to force relogin


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type              | Params
----------------------------------------------------
0 | features      | Sequential        | 6.5 M 
1 | avgpool       | AdaptiveAvgPool2d | 0     
2 | bowel         | Linear            | 1.3 K 
3 | extravasation | Linear            | 1.3 K 
4 | kidney        | Linear            | 3.8 K 
5 | liver         | Linear            | 3.8 K 
6 | spleen        | Linear            | 3.8 K 
7 | loss_module   | BCEWithLogitsLoss | 0     
----------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
26.109    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Monitored metric val_loss did not improve in the last 7 records. Best score: 0.870. Signaling Trainer to stop.
Epoch 7, global step 3272: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 8 records. Best score: 0.870. Signaling Trainer to stop.
Epoch 8, global step 3681: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 9 records. Best score: 0.870. Signaling Trainer to stop.
Epoch 9, global step 4090: 'val_loss' was not in top 1
